In [4]:
import os
import pandas as pd
import numpy as np
from math import ceil, floor

# Translate the Output of the network calculus model to omnetpp xml configuration.
## Input (Network Calculus Model Output)
```
-----  1 . Flow Registration -----
Flow Details: Source  22 , Destination  44 , Bits  1856 , Interval:  2.500 ms , Deadline:  1.000 ms , Priority 1
Path (from, to, prio):  [(22, 2, 1), (2, 44, 1)]
Flow Delay: 1.017ms
Hop (from, to): slope for highest prio, slope for second highest prio, ...
(2, 44) in bit/s: None, 2139234.67, None
```
## Output (OMNeT++ XML Configuration)
```
<ressourceReservation id="1" name="F1 to subscriber N[3]">
    <switchPortIdleSlope switch_id="0A-00-00-00-00-C1" switch_port="3" queue_pcp="6" idle_slope_bps="2139235" />
</ressourceReservation>
```

## Mapping Configurations

In [15]:
nodes= {
    'lidar_fl': "lidarFrontLeft",
    'connectivity_gw': "connectivityGateway",
    'zc_fl': "zonalControllerFrontLeft",
    'infotainment': "infotainment",
    'camera_front': "cameraFront",
    'lidar_fr': "lidarFrontRight",
    'zc_fr': "zonalControllerFrontRight",
    'camera_rear': "cameraRear",
    'lidar_rl': "lidarRearLeft",
    'zc_rl': "zonalControllerRearLeft",
    'adas': "adas",
    'zc_rr': "zonalControllerRearRight",
    'lidar_rr': "lidarRearRight",
}
switches= {
    'sw_fl': "switchFrontLeft",
    'sw_fr': "switchFrontRight",
    'sw_rl': "switchRearLeft",
    'sw_rr': "switchRearRight",
    'sw_c': "switchCenter",
}
devices = nodes.copy()
devices.update(switches)
switchIds= {
    "switchFrontLeft": '0A-AA-00-00-00-06',
    "switchFrontRight": '0A-AA-00-00-00-0C',
    "switchRearLeft": '0A-AA-00-00-00-11',
    "switchRearRight": '0A-AA-00-00-00-16',
    "switchCenter": '0A-AA-00-00-00-1B',
}
links= {
    "switchFrontLeft:0":"switchCenter:0",
    "switchCenter:0":"switchFrontLeft:0",
    "switchCenter:1":"switchFrontRight:0",
    "switchCenter:2":"switchRearRight:0",
    "switchCenter:3":"switchRearLeft:0",
    "switchFrontRight:0":"switchCenter:1",
    "switchRearRight:0":"switchCenter:2",
    "switchRearLeft:0":"switchCenter:3",
    "lidarFrontLeft:0":"switchFrontLeft:1",
    "connectivityGateway:0":"switchFrontLeft:2",
    "zonalControllerFrontLeft:0":"switchFrontLeft:3",
    "infotainment:0":"switchFrontLeft:4",
    "cameraFront:0":"switchFrontRight:1",
    "lidarFrontRight:0":"switchFrontRight:2",
    "zonalControllerFrontRight:0":"switchFrontRight:3",
    "cameraRear:0":"switchRearLeft:1",
    "lidarRearLeft:0":"switchRearLeft:2",
    "zonalControllerRearLeft:0":"switchRearLeft:3",
    "adas:0":"switchRearRight:1",
    "zonalControllerRearRight:0":"switchRearRight:2",
    "lidarRearRight:0":"switchRearRight:3",
}

pcpShift = -2

pcpIdleSlopesBaseDict= {
    7: None,
    6: None,
    5: None,
    4: None,
    3: None,
    2: None,
    1: None,
    0: None
}
# also add all reverse links to the links dictionary
reverseLinks = dict()
for key in links:
    reverseLinks[links[key]] = key
links.update(reverseLinks)

def buildLinkLookup(deviceName, port):
    return deviceName + ":" + str(port)

def splitLinkLookup(link):
    split = link.split(":")
    port = split[1]
    deviceName = split[0]
    return deviceName, port

# returns the device1 device:port and device2 device:port for the link
def getLink(device1, device2):
    for key in links:
        if device1 in key and device2 in links[key]:
            return key, links[key]
    return None, None

# returns the fromDevice:port and toDevice:port for the link
def parseNCLink(link):
    # remove "(" and  ")" from the link
    split = link.replace("(", "").replace(")", "").replace(" ", "").split(",")
    fromDevice = devices[split[0]]
    toDevice = devices[split[1]]
    return getLink(fromDevice, toDevice)

def createSwitchPortIdleSlope(switch, port, pcp, idleSlope):
    global switchIds
    # build an xml string like this <switchPortIdleSlope switch_id="0A-00-00-00-00-C0" switch_port="0" queue_pcp="7" idle_slope_bps="41061186" />
    xml = "<switchPortIdleSlope switch_id=\"" + switchIds[switch] + "\" switch_port=\"" + port + "\" queue_pcp=\"" + pcp + "\" idle_slope_bps=\"" + idleSlope + "\" />"
    return xml

rrIndex = 0
def createResourceReservationBeginIncrementId(name):
    global rrIndex
    rrIndex = rrIndex + 1
    return createResourceReservationBegin(str(rrIndex), name)

def createResourceReservationBegin(id, name):
    # build an xml string like this <resourceReservation id="1" name="F1 to subscriber N[3]">
    return "<resourceReservation id=\"" + id + "\" name=\"" + name + "\">"

def createResourceReservationEnd():
    # build an xml string like this </resourceReservation>
    return "</resourceReservation>"

def createResourceReservationTableBegin():
    # build an xml string like this <resourceReservationTable>
    return "<resourceReservationTable>"

def createResourceReservationTableEnd():
    # build an xml string like this </resourceReservationTable>
    return "</resourceReservationTable>"

def lineIsResourceReservation(line):
    return line[0] == '('

def lineIsFlowDetails(line):
    return "Flow Details" in line

def lineGetFlowName(line):
    # a line looks like this: Flow Details: Source  11 , Destination  33 , Bits  3216 , Interval:  0.125 ms , Deadline:  1.000 ms , Priority 0
    details = line.replace("Flow Details:","").split(",")
    src = ""
    dst = ""
    pcp = ""
    bytes = ""
    interval = ""
    deadline = ""
    for detail in details:
        if "Source" in detail:
            src = nodes[detail.split(":")[-1].strip()]
        if "Destination" in detail:
            # destination is now a list Destination:  ['zc_rr', 'zc_fl']
            # split devices, lookup name in nodes and join them to a list again
            split = detail.split(":")[-1].replace("[","").replace("]","").replace("'","").strip().split(",")
            dst = [nodes[device] for device in split]
        if "Priority" in detail:
            pcp = str(pcpShift + 7-int(detail.split(":")[-1].strip()))
        if "Bits" in detail:
            bytes = str(int(int(detail.split(":")[-1].strip())/8))
        if "Interval" in detail:
            interval = detail.split(":")[-1].strip()
        if "Deadline" in detail:
            deadline = detail.split(":")[-1].strip()
    return "From " + src + " to " + str(dst) + " with PCP " + pcp + " and " + bytes + " bytes in " + interval + " with deadline " + deadline 


def lineGetFlowID(line):
    details = line.replace("Flow Details:","").split(",")

def lineGetLink(line):
    # a line looks like this: (1, 2) in bit/s: 41061186.35, None, None
    link = line.split(")")[0]
    link = link.strip()
    return parseNCLink(link)

def lineGetPcpIdleSlopes(line):
    # a line looks like this: (1, 2) in bit/s: 41061186.35, None, None
    split = line.split(":")[1]
    split = split.replace(" ", "")
    # the first value belongs to the highest priority queue (PCP 7)
    pcp = 7
    idleSlopes = pcpIdleSlopesBaseDict.copy()
    for bandwidth in split.split(","):
        if "None" not in bandwidth:
            idleSlopes[pcp] = int(ceil(float(bandwidth)))
        pcp = pcp - 1
    return idleSlopes

def removeUnchangedIdleSlopes(nextIdleSlopes, previousIdleSlopes):
    # remove all idle slopes that have not changed
    changeIdleSlopes = pcpIdleSlopesBaseDict.copy()
    for pcp in nextIdleSlopes:
        if nextIdleSlopes[pcp] != previousIdleSlopes[pcp]:
            changeIdleSlopes[pcp] = nextIdleSlopes[pcp]
    return changeIdleSlopes

def lineIsFlow(line):
    return "Flow(" in line

def lineGetFlowID(line):
    return int(line.split(",")[0].split("=")[1])

## Transform an actual file

In [16]:
ncOutputFilePath = "dyrectsnOutput10Gig.md"
xmlConfigFilePath = "controllerDyrectsn10GigConfig.xml"

ignoreFlows = [2111, 2112, 2113, 2114]

rrIndex = 0
firstConfig = True
configuredIdleSlopes = dict()
# read the ncOutputFilePath line by line
nextOperationIsRemove = False
skipNextOperation = False
with open(ncOutputFilePath) as ncOutput:
    with open(xmlConfigFilePath, "w") as xmlConfig:
        printString = createResourceReservationTableBegin()
        print(printString)
        xmlConfig.write(printString + "\n")
        for line in ncOutput:
            if "Operation -- remove" in line: 
                nextOperationIsRemove = True
                continue
            if "Operation -- add" in line: 
                nextOperationIsRemove = False
                continue
            if lineIsFlow(line):
                flowID = lineGetFlowID(line)
                if flowID in ignoreFlows:
                    print("Ignoring flow " + str(flowID))
                    skipNextOperation = True
                    continue
                else:
                    skipNextOperation = False
            if nextOperationIsRemove or skipNextOperation:
                continue
            if lineIsFlowDetails(line):
                if not firstConfig:
                    printString = "    " + createResourceReservationEnd()
                    print(printString)
                    xmlConfig.write(printString + "\n")
                name = lineGetFlowName(line)
                printString = "    " + createResourceReservationBeginIncrementId(name)
                print(printString)
                xmlConfig.write(printString + "\n")
                firstConfig = False
                continue
            if lineIsResourceReservation(line):
                fromDevice, toDevice = lineGetLink(line)
                if fromDevice not in configuredIdleSlopes:
                    configuredIdleSlopes[fromDevice] = pcpIdleSlopesBaseDict.copy()
                idleSlopes = lineGetPcpIdleSlopes(line)
                changedIdleSlopes = removeUnchangedIdleSlopes(idleSlopes, configuredIdleSlopes[fromDevice])
                configuredIdleSlopes[fromDevice] = idleSlopes
                for pcp in changedIdleSlopes:
                    if changedIdleSlopes[pcp] is not None:
                        device, port = splitLinkLookup(fromDevice)
                        printString = "        " + createSwitchPortIdleSlope(device, port, str(pcpShift + pcp), str(changedIdleSlopes[pcp]))
                        print(printString)
                        xmlConfig.write(printString + "\n")
        printString = "    " + createResourceReservationEnd()
        print(printString)
        xmlConfig.write(printString + "\n")
        printString = createResourceReservationTableEnd()
        print(printString)
        xmlConfig.write(printString + "\n")


<resourceReservationTable>
    <resourceReservation id="1" name="From zonalControllerFrontRight to ['infotainment'] with PCP 4 and 86 bytes in 200.000 ms with deadline 1.000 ms">
        <switchPortIdleSlope switch_id="0A-AA-00-00-00-06" switch_port="4" queue_pcp="4" idle_slope_bps="3664688" />
        <switchPortIdleSlope switch_id="0A-AA-00-00-00-1B" switch_port="0" queue_pcp="4" idle_slope_bps="3850197" />
        <switchPortIdleSlope switch_id="0A-AA-00-00-00-0C" switch_port="0" queue_pcp="4" idle_slope_bps="3459568" />
    </resourceReservation>
    <resourceReservation id="2" name="From zonalControllerFrontRight to ['zonalControllerRearRight'] with PCP 4 and 86 bytes in 50.000 ms with deadline 1.000 ms">
        <switchPortIdleSlope switch_id="0A-AA-00-00-00-1B" switch_port="2" queue_pcp="4" idle_slope_bps="3850192" />
        <switchPortIdleSlope switch_id="0A-AA-00-00-00-0C" switch_port="0" queue_pcp="4" idle_slope_bps="6895339" />
        <switchPortIdleSlope switch_id="0A-AA-